<a href="https://colab.research.google.com/github/elaynecarvalhos/AEDII/blob/main/autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import plotly.graph_objects as go
import plotly.express as px
import re
import time

#Código disponibilizado pelo professor
class AVLNode:
    """
    A class representing a node in an AVL (Adelson-Velsky and Landis) tree,
    which is a self-balancing binary search tree.

    Attributes:
        value: The value (word) stored in the node.
        left_child (AVLNode): The left child node.
        right_child (AVLNode): The right child node.
        height (int): The height of the subtree rooted at this node.
        imbalance (int): The imbalance factor of this node, calculated as
                         the difference between the heights of the left and
                         right subtrees.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value (str): The value (word) to be stored in this node.
        """
        self.value = value
        self.left_child = None
        self.right_child = None
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        """
        Calculates the height and imbalance factor of this node based
        on the heights of its left and right children.

        This method assumes that the heights of the children nodes (if they exist)
        are up-to-date.
        """
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        self.height = 1 + max(left_height, right_height)
        self.imbalance = left_height - right_height


class AVLTree:
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.

    Attributes:
        root (AVLNode or None): The root node of the AVL tree. Initializes to None for an empty tree.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        self.root = None

    def add(self, value):
        """
        Inserts a new value into the AVL Tree and rebalances the tree as needed.

        Args:
            value (str): The value (word) to be added to the tree.
        """
        self.root = self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        Recursively finds the correct position and inserts a value into the AVL tree.
        This method also ensures the tree remains balanced by updating node heights and performing rotations as needed.

        Args:
            current_node (AVLNode): The node to start the search for the insert position from.
            value (str): The value (word) to be added to the AVL tree.

        Returns:
            AVLNode: The node that either gets inserted or the node that was already present at that position.
        """

        if current_node is None:
            return AVLNode(value)

        if value < current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        current_node.calculate_height_and_imbalance()

        if abs(current_node.imbalance) == 2:
            return self._balance(current_node)

        return current_node

    def _rotate_left(self, node):
        """
        Performs a left rotation on the given node and adjusts the height and imbalance attributes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """
        pivot = node.right_child
        node.right_child = pivot.left_child
        pivot.left_child = node

        node.calculate_height_and_imbalance()
        pivot.calculate_height_and_imbalance()

        return pivot

    def _rotate_right(self, node):
        """
        Performs a right rotation on the given node and adjusts the height and imbalance attributes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """
        pivot = node.left_child
        node.left_child = pivot.right_child
        pivot.right_child = node

        node.calculate_height_and_imbalance()
        pivot.calculate_height_and_imbalance()

        return pivot

    def _balance(self, node):
        """
        Balances the subtree rooted at the given node by performing rotations as needed.

        Args:
            node (AVLNode): The root node of the subtree that needs to be balanced.

        Returns:
            AVLNode: The new root node of the balanced subtree.

        Note:
            This method assumes that the height and imbalance factor of each node are up-to-date.
        """
        if node.imbalance == 2:
            pivot = node.left_child
            if pivot.imbalance == 1:
                return self._rotate_right(node)
            else:
                node.left_child = self._rotate_left(pivot)
                return self._rotate_right(node)
        else:
            pivot = node.right_child
            if pivot.imbalance == -1:
                return self._rotate_left(node)
            else:
                node.right_child = self._rotate_right(pivot)
                return self._rotate_left(node)

    def clean_text(self, text):
        """
        Removes punctuation and special characters from a text.

        Args:
            text (str): The text to be cleaned.

        Returns:
            str: The text without punctuation and special characters.
        """
        cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
        return cleaned_text

    def insert_unique_words(self, text):
        """
        Inserts unique words from a text into the AVL tree.

        Args:
            text (str): The text containing words to be inserted.
        """
        text = text.lower()
        cleaned_text = self.clean_text(text)
        words = cleaned_text.split()
        unique_words = set(words)
        for words in unique_words:
            self.add(words)

    def find_words_with_prefix(self, prefix):
        """
        Finds words in the AVL tree that start with a given prefix.

        Args:
            prefix (str): The prefix to search for.

        Returns:
            list: A list of words that start with the given prefix.
        """
        prefix = self.clean_text(prefix).lower()
        completions = []
        self._find_words_with_prefix_recursive(self.root, prefix, completions)
        return completions

    def _find_words_with_prefix_recursive(self, node, prefix, completions):
        """
        Recursively finds words in the AVL tree that start with a given prefix.

        Args:
            node (AVLNode): The current node being checked.
            prefix (str): The prefix to search for.
            completions (list): The list to store found completions.
        """
        if node is None:
            return

        if node.value.lower().startswith(prefix):
            completions.append(node.value)

        if node.value.lower() >= prefix:
            self._find_words_with_prefix_recursive(node.left_child, prefix, completions)

        if node.value.lower() <= prefix:
            self._find_words_with_prefix_recursive(node.right_child, prefix, completions)



In [37]:
# Função para inserção das palavras em uma lista
def insert_words_in_list(text):
    text = text.lower()
    cleaned_text = avl_tree.clean_text(text)
    words = cleaned_text.split()
    unique_words = set(words)
    return list(unique_words)

In [38]:
# Instância da árvore AVL
avl_tree = AVLTree()

In [39]:
# Corpus "Astronômia"
words= """
A astronomia é a ciência que estuda os corpos celestes e os fenômenos que ocorrem no universo além da atmosfera da Terra. Desde tempos antigos, a humanidade olha para o céu em busca de respostas sobre sua origem, seu lugar no cosmos e seu destino. Astronomia é uma disciplina vasta, que abrange uma gama impressionante de tópicos, desde o estudo das estrelas e planetas até a investigação de galáxias distantes e as leis fundamentais do universo.
Os astrônomos observam o céu usando telescópios terrestres e espaciais, detectando e analisando radiações eletromagnéticas em várias faixas do espectro, como luz visível, infravermelho, ultravioleta e ondas de rádio. Eles exploram a física dos corpos celestes, suas órbitas, composições químicas e evolução ao longo do tempo.
Além disso, a astronomia desempenha um papel fundamental na compreensão das origens da vida e na busca por outros mundos habitáveis fora do nosso sistema solar, na chamada busca por exoplanetas. Ela também está intrinsecamente ligada à astrofísica, que estuda os processos físicos que ocorrem nos corpos celestes, como a fusão nuclear nas estrelas e os buracos negros.
Graças aos avanços tecnológicos, como telescópios espaciais e supercomputadores, os astrônomos são capazes de fazer descobertas incríveis, como a detecção de buracos negros colidindo, a identificação de novos planetas em sistemas solares distantes e a medição da expansão do universo.
Em resumo, a astronomia continua a desempenhar um papel vital na expansão do nosso conhecimento sobre o universo, inspirando a curiosidade humana e promovendo o progresso científico em uma jornada contínua para desvendar os mistérios do cosmos."""


In [40]:
# Remoção de pontuação e caracteres especiais do corpus
words = words.lower()

cleaned_corpus = avl_tree.clean_text(words)

In [41]:
# Dividindo o corpus em palavras individuais
corpus_words = cleaned_corpus.split()

# Removendo palavras duplicadas
corpus_words = list(dict.fromkeys(corpus_words))

In [48]:

# Inserindo todas as palavras únicas do corpus na árvore AVL e medindo o tempo de inserção
start_time = time.time()
avl_tree.insert_unique_words(words)
end_time = time.time()
print("Tempo de Inserção na Árvore AVL:", end_time - start_time, "segundos")

# Inserindo todas as palavras únicas do corpus em uma lista e medindo o tempo de inserção
start_time = time.time()
unique_words_list = insert_words_in_list(words)
end_time = time.time()
print("Tempo de Inserção na Lista:", end_time - start_time, "segundos")

# Comparando a altura da árvore AVL com o tamanho da lista
print("Altura da Árvore AVL:", avl_tree.root.height)
print("Tamanho da Lista:", len(unique_words_list))

# Encontrando palavras com um determinado prefixo na árvore AVL
prefix = "c"
start_time = time.time()
completions_avl = avl_tree.find_words_with_prefix(prefix)
end_time = time.time()
print(f"Tempo de Busca com Árvore AVL (Prefixo '{prefix}'): {end_time - start_time} segundos")
print(f"Palavras com o Prefixo '{prefix}' na árvore AVL: ")
print(completions_avl)

# Encontrando palavras com um determinado prefixo na lista
prefix = "c"
start_time = time.time()
completions_list = [word for word in unique_words_list if word.startswith(prefix)]
end_time = time.time()
print(f"Tempo de Busca com Lista (Prefixo '{prefix}'): {end_time - start_time} segundos")
print(f"Palavras com o Prefixo '{prefix}' na lista:")
print(completions_list)

Tempo de Inserção na Árvore AVL: 0.003951072692871094 segundos
Tempo de Inserção na Lista: 0.00030803680419921875 segundos
Altura da Árvore AVL: 11
Tamanho da Lista: 153
Tempo de Busca com Árvore AVL (Prefixo 'c'): 0.00012540817260742188 segundos
Palavras com o Prefixo 'c' na árvore AVL: 
['contnua', 'chamada', 'capazes']
Tempo de Busca com Lista (Prefixo 'c'): 0.00016617774963378906 segundos
Palavras com o Prefixo 'c' na lista:
['contnua', 'cientfico', 'cosmos', 'capazes', 'chamada', 'corpos', 'cincia', 'curiosidade', 'conhecimento', 'composies', 'celestes', 'como', 'cu', 'continua', 'colidindo', 'compreenso']


In [49]:
print(corpus_words)

['a', 'astronomia', 'cincia', 'que', 'estuda', 'os', 'corpos', 'celestes', 'e', 'fenmenos', 'ocorrem', 'no', 'universo', 'alm', 'da', 'atmosfera', 'terra', 'desde', 'tempos', 'antigos', 'humanidade', 'olha', 'para', 'o', 'cu', 'em', 'busca', 'de', 'respostas', 'sobre', 'sua', 'origem', 'seu', 'lugar', 'cosmos', 'destino', 'uma', 'disciplina', 'vasta', 'abrange', 'gama', 'impressionante', 'tpicos', 'estudo', 'das', 'estrelas', 'planetas', 'at', 'investigao', 'galxias', 'distantes', 'as', 'leis', 'fundamentais', 'do', 'astrnomos', 'observam', 'usando', 'telescpios', 'terrestres', 'espaciais', 'detectando', 'analisando', 'radiaes', 'eletromagnticas', 'vrias', 'faixas', 'espectro', 'como', 'luz', 'visvel', 'infravermelho', 'ultravioleta', 'ondas', 'rdio', 'eles', 'exploram', 'fsica', 'dos', 'suas', 'rbitas', 'composies', 'qumicas', 'evoluo', 'ao', 'longo', 'tempo', 'disso', 'desempenha', 'um', 'papel', 'fundamental', 'na', 'compreenso', 'origens', 'vida', 'por', 'outros', 'mundos', 'habitv

###Análise de Desempenho:

1) Compare o desempenho (tempo de inserção, altura da árvore, tempo de busca)
da árvore AVL com uma estrutura de dados mais simples, como uma lista ou
uma árvore binária de busca sem balanceamento.

  - Tempo de Inserção:

Percebe-se que o tempo de inserção da árvore AVL é mais lento comparando com o tempo de inserção da lista ou uma árvore binária de busca simples.
  - Altura da Árvore:

Notou-se que a altura da árvore AVL é mais baixa e mais previsível em comparação com uma árvore binária de busca simples, visto que a lista não possui altura.

  - Tempo de Busca:

Observou-se também que o tempo de busca na árvore AVL é mais rápido do que uma lista não ordenada.

2) Impacto do tamanho do corpus no desempenho da árvore AVL:

A árvore AVL é uma estrutura de dados eficaz para inserção e pesquisa, mesmo em textos extensos, devido ao seu balanceamento automático. No entanto, o desempenho real pode variar dependendo da distribuição das palavras no texto e da eficiência da implementação. Em conjuntos de dados muito grandes, outras estruturas, como árvores B ou tabelas de dispersão (hash tables), podem ser mais adequadas para otimização adicional.